In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt

root_path = '/var/datasets/single_papers/2023_feargal/GSE199750'


In [ ]:
metadata_df = read_prefixes_from_metadata_txt(f'{root_path}/GSE199750_series_matrix.txt')
print(metadata_df.shape)
metadata_df.head()

In [ ]:
input_dir = f'{root_path}/GSE199750_RAW'
count_matrix = pd.DataFrame()


for file in os.listdir(input_dir):
    if file.endswith(".featureCounts.txt"):
        file_path = os.path.join(input_dir, file)
        data = pd.read_csv(file_path, comment='#', sep='\t')
        gene_counts = data[['Geneid', data.columns[-1]]]  # Last column is the count column
        sample_name = next(part for part in file.split('_') if part.startswith("GSM"))
        gene_counts.columns = ['Geneid', sample_name]
        if count_matrix.empty:
            count_matrix = gene_counts
        else:
            count_matrix = pd.merge(count_matrix, gene_counts, on='Geneid', how='outer')
count_matrix_T = count_matrix.T

        

In [ ]:
count_matrix_T.columns = count_matrix_T.iloc[0]
count_matrix_T = count_matrix_T[1:]
#count_matrix_T.reset_index(drop=True, inplace=True)
print(count_matrix_T.shape)
count_matrix_T.head()

In [ ]:
# As I only want the pre vaccine expression counts: 

metadata_v0_df = metadata_df[metadata_df['title'].str.contains('V0')]
print(metadata_v0_df.shape)
metadata_v0_df.tail()

In [ ]:
combi_df = count_matrix_T.merge(metadata_v0_df, right_on='geo_accession', how='inner', left_index=True)
print(combi_df.shape)
combi_df.head()


In [ ]:
combi_df.to_csv('/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv')